### Load data

In [116]:
import pandas as pd
books_data = pd.read_csv('data/books_data.csv')
books_ratings = pd.read_csv('data/books_rating.csv')

In [117]:
# Titles
books_data_1 = books_data.drop(['image', 'previewLink', 'infoLink', 'publisher', 'ratingsCount'], axis=1, inplace=False)
books_ratings_1 = books_ratings.drop(['Id','Price', 'profileName', 'review/time'], axis=1, inplace=False)

books_data_1.rename(columns={'Title': 'title','publishedDate': 'published_date'}, inplace=True)
books_ratings_1.rename(columns={'Title': 'title','User_id':'user_id',  'review/score': 'score','review/helpfulness': 'helpfulness', 'review/text': 'text', 'review/summary': 'summary'}, inplace=True)

print(list(books_data_1.columns))
print(list(books_ratings_1.columns))

['title', 'description', 'authors', 'published_date', 'categories']
['title', 'user_id', 'helpfulness', 'score', 'summary', 'text']


In [118]:
books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: s.lower() if type(s) == str else s)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"\(.*\)","", regex=True)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"\&","and", regex=True)


books_data_1['title'] = books_data_1['title'].map(lambda s: s.lower() if type(s) == str else s)
books_data_1['title'] = books_data_1['title'].replace(r"\&","and", regex=True)
books_data_1['title'] = books_data_1['title'].replace(r"\(.*\)","", regex=True)

# remove all special characters with regex
books_ratings_1['title'] = books_ratings_1['title'].replace(r"[^a-zA-Z0-9\s]+", "", regex=True)
books_data_1['title'] = books_data_1['title'].replace(r"[^a-zA-Z0-9\s]+", "", regex=True)

books_data_1['title'] = books_data_1['title'].replace(r"[ \t]+$","", regex=True)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"[ \t]+$","", regex=True)


In [119]:
# keep only unique titles in books_data_1
books_data_1 = books_data_1.drop_duplicates(subset=['title'], keep='first')

In [120]:
#books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'the hobbit' if (type(s) == str and 'hobbit' in s) else s)
#books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'lord of the rings' if (type(s) == str and 'lord of the rings' in s) else s)
#books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'pride and prejudice' if (type(s) == str and 'pride & prejudice' in s) else s)
#books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'pride and prejudice' if (type(s) == str and 'pride and prejudice' in s) else s)
#books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'the great gatsby' if (type(s) == str and 'great gatsby' in s) else s)
#books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'to kill a mocking bird' if (type(s) == str and 'mocking bird' in s) else s)
#books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'to kill a mocking bird' if (type(s) == str and 'mockingbird' in s) else s)

#books_data_1['title'] = books_data_1['title'].map(lambda s: 'the hobbit' if (type(s) == str and 'hobbit' in s) else s)
#books_data_1['title'] = books_data_1['title'].map(lambda s: 'lord of the rings' if (type(s) == str and 'lord of the rings' in s) else s)
#books_data_1['title'] = books_data_1['title'].map(lambda s: 'pride and prejudice' if (type(s) == str and 'pride & prejudice' in s) else s)
#books_data_1['title'] = books_data_1['title'].map(lambda s: 'pride and prejudice' if (type(s) == str and 'pride and prejudice' in s) else s)
#books_data_1['title'] = books_data_1['title'].map(lambda s: 'the great gatsby' if (type(s) == str and 'great gatsby' in s) else s)
#books_data_1['title'] = books_data_1['title'].map(lambda s: 'to kill a mocking bird' if (type(s) == str and 'mocking bird' in s) else s)
#books_data_1['title'] = books_data_1['title'].map(lambda s: 'to kill a mocking bird' if (type(s) == str and 'mockingbird' in s) else s)

In [121]:
# load book data for books in book ratings
#ratings_titles = books_ratings_1['title'].unique()
#books_data_1 = books_data_1[books_data_1['title'].isin(ratings_titles)]

In [122]:
n_ratings = books_ratings_1.groupby('title').count()['score']>=10
famous_books = n_ratings[n_ratings].index
books_ratings_1 = books_ratings_1[books_ratings_1['title'].isin(famous_books)]

n_ratings_1 = books_ratings_1['title'].value_counts().rename("n_ratings")

books_data_1 = books_data_1.merge(n_ratings_1, on='title', how='inner')
print(books_data_1.columns)


Index(['title', 'description', 'authors', 'published_date', 'categories',
       'n_ratings'],
      dtype='object')


In [123]:
# remove if n_ratings < 10
books_data_1 = books_data_1[books_data_1['n_ratings']>=10]

In [124]:
x = books_ratings_1.groupby('user_id').count()['score'] > 20 # since this is what amazon requires before a user can get recommendations
considerable_users = x[x].index
books_ratings_1 = books_ratings_1[books_ratings_1['user_id'].isin(considerable_users)]
#
# print(books_ratings_1.sample(5))

books_ratings_1['user_id'].value_counts()

user_id
A1D2C0WDCSHUWZ    3003
AFVQZQ8PW0L       2447
A14OJS0VWMOSWO    2412
A1X8VZWTOG8IS6    1438
AHD101501WCN1     1228
                  ... 
A2QHCXMFJPEIS1      21
A2JFDCLA9PZGQ0      21
A1RZFX5PRKEXVE      21
A2OW356OMJNAVH      21
A30KOED2O09QJS      21
Name: count, Length: 7766, dtype: int64

In [125]:
books_ratings_1['title'].value_counts()

title
wuthering heights                                                                        8048
pride and prejudice                                                                      7166
the picture of dorian gray                                                               4716
little women or meg jo beth and amy                                                      3294
emma                                                                                     3135
                                                                                         ... 
staying sober a guide for relapse prevention based upon the cenaps model of treatment       1
the world rushed in the california gold rush experience                                     1
the naive and sentimental lover                                                             1
charms and charm bracelets the complete guide                                               1
smalltalk best practice patterns                      

In [126]:
# extract all books rated by A14OJS0VWMOSWO
books_ratings_1[books_ratings_1['user_id'] == 'A14OJS0VWMOSWO']['title'].value_counts()

title
alices adventures in wonderland         21
the white company                       15
great expectations                      12
ulysses                                 11
alice in wonderland                      9
                                        ..
trick is in the training the             1
becoming chloe                           1
carfree cities                           1
firestarter                              1
premiere 6 for macintosh and windows     1
Name: count, Length: 1995, dtype: int64

In [127]:
avg_rating = books_ratings_1.groupby('title')["score"].mean().rename("avg_rating")
books_data_1 = books_data_1.merge(avg_rating, on='title', how='inner')
#print(books_data_1.columns)


In [128]:
def get_n_helpful(x):
    if isinstance(x, str) and '/' in x:
        num, denom = x.split('/')
        return int(denom)
    else:
        return x

def replace_fraction(x):
    if isinstance(x, str) and '/' in x:
        num, denom = x.split('/')
        if denom == '0':
            return 0
        else:
            return int(num) / int(denom)
    else:
        return x

books_ratings_1['helpfulness_count'] = books_ratings_1['helpfulness'].apply(get_n_helpful)
books_ratings_1['helpfulness_pct'] = books_ratings_1['helpfulness'].apply(replace_fraction)

#print(books_ratings_1.sample(2))

In [129]:
#print(books_data_1.columns)

In [130]:
#def extract_year(x):
#    year1 = pd.to_datetime(x, format='mixed').year
##    print(year1)
#    return year1
    
#books_data_1['published_date'] = books_data_1['published_date'].apply(extract_year)
#p#rint(books_data_1.sample(5))


In [131]:
import os
if not os.path.exists('clean_data'):
    os.makedirs('clean_data')

# save clean data
books_data_1.to_csv('clean_data/books_data_clean.csv')
books_ratings_1.to_csv('clean_data/books_rating_clean.csv')

print("First we had {} books and {} ratings".format(len(books_data), len(books_ratings)))
print("Now we have {} books and {} ratings".format(len(books_data_1), len(books_ratings_1)))

First we had 212404 books and 3000000 ratings
Now we have 32102 books and 435110 ratings
